In [15]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from scipy.misc import toimage
import skimage.color

device = torch.device('cuda')     # Default CUDA device

In [ ]:
# model hyper parameters

ALPHA = 1000
LAMBDA = 10
BETA1 =0.5
BETA2 = 0.999
EPOCHS = 5

In [40]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        #  Convolutional layers 
        
        # input 512x512x3  output 512x512x16
        self.conv1 = nn.Conv2d(3, 16, 5, stride = 1, padding = 1)
        self.conv1_bn = nn.BatchNorm2d(16)
        
        # input 512x512x16  output 256x256x32
        self.conv2 = nn.Conv2d(16, 32, 5, stride = 2, padding = 1)
        self.conv2_bn = nn.BatchNorm2d(32)
        
        # input 265x256x32  output 128x128x64
        self.conv3 = nn.Conv2d(32, 64, 5, stride = 2, padding = 1)
        self.conv3_bn = nn.BatchNorm2d(64)
        
        # input 128x128x64  output 64x64x128
        self.conv4 = nn.Conv2d(64, 128, 5, stride = 2, padding = 1)
        self.conv4_bn = nn.BatchNorm2d(128)
        
        # input 64x64x128  output 32x32x128
        # the output of this layer we need layers for global features
        self.conv5 = nn.Conv2d(128, 128, 5, stride = 2, padding = 1)
        self.conv5_bn = nn.BatchNorm2d(128)
        
        # convs for global features
        # input 32x32x128 output 16x16x128
        self.conv51 = nn.Conv2d(128,128,5, stride =2 , padding =1 )
        
        # input 16x16x128 output 8x8x128
        self.conv52 = nn.Conv2d(128,128,5, stride =2 , padding =1 )
        
        # input 8x8x128 output 1x1x128
        self.conv531 = nn.Conv2d(128,128,5, stride =2 , padding =1 )
        
        # input 1x1x128 output 1x1x128
        self.conv532 = nn.Conv2d(128,128,5, stride =2 , padding =1 )
        
        # input 32x32x128 output 32x32x128
        # the global features should be concatenated to the feature map aftere this layer
        # the output after concat would be 32x32x256
        self.conv6 = nn.Conv2d(128, 128, 5, stride = 1, padding = 1)
        
        # input 32x32x256 output 32x32x128
        self.conv7 = nn.Conv2d(256, 128, 5, stride = 1, padding = 1)
        
        # deconvolutional layers
        # input 32x32x128 output 64x64x128
        self.dconv1 = nn.ConvTranspose2d(128, 128, 5, stride = 2, padding = 1)
        self.dconv1_bn = nn.BatchNorm2d(128)
        
        # input 64x64x256 ouput 128x128x128
        self.dconv2 = nn.ConvTranspose2d(256, 128, 5, stride = 2, padding = 1)
        self.dconv2_bn = nn.BatchNorm2d(128)
        
        # input 128x128x192 output 256x256x64
        self.dconv3 = nn.ConvTranspose2d(192, 64, 5, stride = 2, padding = 1)
        self.dconv3_bn = nn.BatchNorm2d(64)
        
        # input 256x256x96 ouput 512x512x32
        self.dconv4 = nn.ConvTranspose2d(96, 32, 5, stride = 2, padding = 1)
        self.dconv4_bn = nn.BatchNorm2d(32)
        
        # final convolutional layers
        # input 512x512x48 output 512x512x16
        self.conv8 = nn.Conv2d(48, 16, 5, stride = 1, padding = 1)
        self.conv8_bn = nn.BatchNorm2d(16)
        
        # input 512x512x16 output 512x512x3
        self.conv9 = nn.Conv2d(16, 3, 5, stride = 1, padding = 1)    
        self.conv9_bn = nn.BatchNorm2d(3)
        # SELU
                
    def forward(self, x):
        # input 512x512x3 to output 512x512x16
        x = self.conv1_bn(F.selu(self.conv1(x)))
        
        # input 512x512x16 to output 256x256x32
        x1 = self.conv2_bn(F.selu(self.conv2(x)))
        
        # input 256x256x32 to output 128x128x64
        x2 = self.conv3_bn(F.selu(self.conv3(x1)))
        
        # input 128x128x64 to output 64x64x128
        x3 = self.conv4_bn(F.selu(self.conv4(x2)))
        
        # input 64x64x128 to output 32x32x128
        x4 = self.conv5_bn(F.selu(self.conv5(x3)))
        
        #convolutions for global features
        
        # input 32x32x128 to output 16x16x128
        x51 = self.conv51(x4)
        # input 16x16x128 to output 8x8x128
        x52 = self.conv52(x51)
        # input 8x8x128 to output 1x1x128
        x53 = self.conv532(F.selu(self.conv531(x52)))
        x53_temp = torch.cat([x53]*32)
        x53_temp = torch.cat([x53_temp]*32,dim=1)
        
        
        # input 32x32x256 to output 32x32x128
        x5 = self.conv6(x4)
        
        # input 32x32x128 to output 32x32x128
        x5 = self.conv7(torch.cat(x5,x53_temp))
        
        # input 32x32x128 to output 64x64x128
        xd = self.dconv1(self.dconv1_bn(F.selu(x5)))
        
        # input 64x64x256 to output 128x128x128
        xd = self.dconv2(self.dconv2_bn(F.selu(torch.cat((xd,x3),dim=1))))
        
        # input 128x128x192 to output 256x256x64
        xd = self.dconv3(self.dconv3_bn(F.selu(torch.cat((xd,x2),dim=1))))
        
        # input 256x256x64 to output 512x512x32
        xd = self.dconv4(self.dconv4_bn(F.selu(torch.cat((xd,x1),dim=1))))
        
        # input 512x512x48 to output 512x512x16
        xd = self.conv8(self.conv8_bn(F.selu(torch.cat((xd,x),dim=1))))
        
        # input 512x512x16 to output 512x512x3
        xd = self.conv9(self.conv9_bn(F.selu((xd))))
                        
                        
        return xd

In [41]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        #  Convolutional layers 
        
        # input 512x512x3  output 512x512x16
        self.conv1 = nn.Conv2d(3, 16, 5, stride = 1, padding = 1)
        self.conv1_in = nn.InstanceNorm2d(16)
        
        # input 512x512x16  output 256x256x32
        self.conv2 = nn.Conv2d(16, 32, 5, stride = 2, padding = 1)
        self.conv2_in = nn.InstanceNorm2d(32)
        
        # input 265x256x32  output 128x128x64
        self.conv3 = nn.Conv2d(32, 64, 5, stride = 2, padding = 1)
        self.conv3_in = nn.InstanceNorm2d(64)
        
        # input 128x128x64  output 64x64x128
        self.conv4 = nn.Conv2d(64, 128, 5, stride = 2, padding = 1)
        self.conv4_in = nn.InstanceNorm2d(128)
        
        # input 64x64x128  output 32x32x128
        # the output of this layer we need layers for global features
        self.conv5 = nn.Conv2d(128, 128, 5, stride = 2, padding = 1)
        self.conv5_in = nn.InstanceNorm2d(128)
        
        # input 32x32x128  output 16x16x128
        # the output of this layer we need layers for global features
        self.conv6 = nn.Conv2d(128, 128, 5, stride = 2, padding = 1)
        self.conv6_in = nn.InstanceNorm2d(128)
        
        # input 16x16x128  output 1x1x1
        # the output of this layer we need layers for global features
        self.conv6 = nn.Conv2d(128, 1, 5, stride = 32, padding = 1)
        self.conv6_in = nn.InstanceNorm2d(1)
        
    def forward(self, x):
        
        # input 512x512x3 to output 512x512x16
        x = self.conv1_in(F.leaky_relu(self.conv1(x)))
        
        # input 512x512x16 to output 256x256x32
        x = self.conv2_in(F.leaky_relu(self.conv2(x)))
        
        # input 256x256x32 to output 128x128x64
        x = self.conv3_in(F.leaky_relu(self.conv3(x)))
        
        # input 128x128x64 to output 64x64x128
        x = self.conv4_in(F.leaky_relu(self.conv4(x)))
        
        # input 64x64x128 to output 32x32x128
        x = self.conv5_in(F.leaky_relu(self.conv5(x)))
        
        # input 32x32x128 to output 16x16x128
        x = self.conv5_in(F.leaky_relu(self.conv5(x)))
        
        # input 16x16x128 to output 1x1x1
        x = self.conv5_in(F.leaky_relu(self.conv5(x)))
        
        return x

In [30]:
# x53 = torch.ones([1, 1, 128], dtype=torch.int32)
# x53_temp = torch.cat([x53]*32)
# x53_temp = torch.cat([x53_temp]*32,dim=1)
# print(x53_temp.shape)
# print(x53_temp)

# Converting the images for PILImage to tensor, so they can be accepted as the input to the network

transform = transforms.Compose([transforms.Resize(512, interpolation=2),
                                transforms.ToTensor()])
trainset = torchvision.datasets.STL10(root='./data', split="unlabeled", download=True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=5, shuffle=False)

testset = torchvision.datasets.STL10(root='./data', split = "test", download=True,
                                     transform =transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified
Dataset STL10
    Number of datapoints: 100000
    Split: unlabeled
    Root Location: ./data
    Transforms (if any): Compose(
                             Resize(size=512, interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                         )
    Target Transforms (if any): None
Dataset STL10
    Number of datapoints: 8000
    Split: test
    Root Location: ./data
    Transforms (if any): Compose(
                             Resize(size=512, interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                         )
    Target Transforms (if any): None
torch.Size([5, 3, 512, 512])


In [31]:
print(trainset)
print(testset)

dataiter = iter(testloader)
images, labels = dataiter.next()
print(images.shape)

Dataset STL10
    Number of datapoints: 100000
    Split: unlabeled
    Root Location: ./data
    Transforms (if any): Compose(
                             Resize(size=512, interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                         )
    Target Transforms (if any): None
Dataset STL10
    Number of datapoints: 8000
    Split: test
    Root Location: ./data
    Transforms (if any): Compose(
                             Resize(size=512, interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                         )
    Target Transforms (if any): None
torch.Size([5, 3, 512, 512])


In [42]:
gen1 = Generator()
gen2 = Generator()
dis = Discriminator()

print(gen1)
print(gen2)
print(dis)

Generator(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv1_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
  (conv2_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
  (conv3_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
  (conv4_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(128, 128, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
  (conv5_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv51): Conv2d(128, 128, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
  (conv52): Conv2d(128, 128, kernel_size=(5, 5), s

In [46]:
# optimiser parameters

gen1_optimizer = optim.Adam( gen1.parameters(), lr = 0.001,  betas = (BETA1, BETA2))
gen2_optimizer = optim.Adam( gen2.parameters(), lr = 0.001,  betas = (BETA1, BETA2))
dis_optimizer = optim.Adam( dis.parameters(), lr = 0.001,  betas = (BETA1, BETA2))


In [47]:
criterion= nn.MSELoss()

#losses
# AG + alpha*I
# alpha = 1000

def gen_adv_loss(input_images, output_images):
    
    # generator adversarial loss
    validity = disc(images)
    gen_adv_loss = torch.mean(validity)
    
    #Identity mapping loss
    i_loss = criterion(input_images,output_images)
    
    total_gen_loss = -gen_adv_loss + ALPHA*i_loss
    
    return total_gen_loss


NameError: name 'cuda' is not defined

In [ ]:
batches_done = 0
for epoch in range(EPOCHS):
    for i, (imgs, _) in enumerate(dataloader):
    
        # Configure input
        real_imgs = Variable(imgs.type(Tensor))